<a href="https://colab.research.google.com/github/darshan-k3/Low-Complexity-Deepfake-Detection-Model/blob/main/Multimodal_Model_Training_and_Evaluation_FINAL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Starting point

In [ ]:
!rm -r /content/.ipynb_checkpoints

rm: cannot remove '/content/.ipynb_checkpoints': No such file or directory


In [ ]:
#@title 0.1 Installs Necessary Python Libraries
import gdown
import os
# update at the end
requirements_txt_url = 'https://drive.google.com/file/d/1yKZtv18wki7O1Ncruv0fclH7lbtvTWiQ/view?usp=drive_link'
if not os.path.isfile('requirements.txt'):
  gdown.download(requirements_txt_url, fuzzy=True)

# %pip install -r requirements.txt

Downloading...
From: https://drive.google.com/uc?id=1yKZtv18wki7O1Ncruv0fclH7lbtvTWiQ
To: /content/requirements.txt
100%|██████████| 234/234 [00:00<00:00, 838kB/s]


In [ ]:
#@title 1.0. Download benchmark dataset, clear files from previous runs

# Make sure Google Drive has been mounted
from google.colab import drive
import gdown
# drive.mount('/content/drive', force_remount=True)

import os
# !rm -r benchmark_dataset
if not os.path.isfile('benchmark_dataset.zip'):
  gdown.download('https://drive.google.com/file/d/1E1QwFJeby_bkRFd2SbIWS9-td3SKqNp6/view?usp=sharing', fuzzy=True)

if not os.path.isdir('benchmark_dataset'):
  !unzip benchmark_dataset.zip

# Remove hidden .ipynb checkpoints folder
if os.path.isdir('benchmark_dataset/RealVideo-FakeAudio/.ipynb_checkpoints'):
  !rm -r benchmark_dataset/RealVideo-FakeAudio/.ipynb_checkpoints
  !rm -r benchmark_dataset/FakeVideo-RealAudio/.ipynb_checkpoints

# Remove any preexisting pretrained models
models = ['scaler_audio.pkl', 'scaler_video.pkl', 'svm_model_audio.pkl', 'svm_model_video.pkl',
          'checkpoint.pt', 'mlp_video_model.keras', 'mlp_audio_model.keras']

base_model_dir = 'models'

for model in models:
  model_path = os.path.join(base_model_dir, model)
  if os.path.isfile(model_path):
    !rm -r model_path


if os.path.isdir('temp'):
  !rm -r temp/
os.makedirs('temp', exist_ok=True)

Downloading...
From (original): https://drive.google.com/uc?id=1E1QwFJeby_bkRFd2SbIWS9-td3SKqNp6
From (redirected): https://drive.google.com/uc?id=1E1QwFJeby_bkRFd2SbIWS9-td3SKqNp6&confirm=t&uuid=ab3b6804-d7a0-434e-b578-4b75058639c3
To: /content/benchmark_dataset.zip
100%|██████████| 206M/206M [00:01<00:00, 119MB/s]


Archive:  benchmark_dataset.zip
   creating: benchmark_dataset/FakeVideo-FakeAudio/
  inflating: benchmark_dataset/FakeVideo-FakeAudio/men_id00018_00181_id00529_wavtolip.mp4  
  inflating: benchmark_dataset/FakeVideo-FakeAudio/men_id00029_00288_id00052_NK_leX996Yk_id00696_wavtolip.mp4  
  inflating: benchmark_dataset/FakeVideo-FakeAudio/men_id00032_00028_1_id07108_wavtolip.mp4  
  inflating: benchmark_dataset/FakeVideo-FakeAudio/men_id00033_00276_id07195_wXqfHGtaBn4_id00032_wavtolip.mp4  
  inflating: benchmark_dataset/FakeVideo-FakeAudio/men_id00049_00118_id00775_yeCUxHgUHxc_id00169_wavtolip.mp4  
  inflating: benchmark_dataset/FakeVideo-FakeAudio/men_id00055_00120_id00415_wavtolip.mp4  
  inflating: benchmark_dataset/FakeVideo-FakeAudio/men_id00063_00021_id00919_XDUn0Pskyrk_id00554_wavtolip.mp4  
  inflating: benchmark_dataset/FakeVideo-FakeAudio/men_id00076_00109_9_id01528_wavtolip.mp4  
  inflating: benchmark_dataset/FakeVideo-FakeAudio/men_id00078_00114_id03599_y27BDmRUqMQ_id07161

In [ ]:
#@title 1.1 Create Testing Dataset (Optional)

# The purpose of this dataset is to debug any problems with the pipeline
# running on larger sets
import shutil
import os
import random

if os.path.isdir('debugging_dataset'):
  !rm -r test_dataset

real_real_dir = os.path.join('benchmark_dataset', 'RealVideo-RealAudio')
real_fake_dir = os.path.join('benchmark_dataset', 'RealVideo-FakeAudio')
fake_real_dir = os.path.join('benchmark_dataset', 'FakeVideo-RealAudio')
fake_fake_dir = os.path.join('benchmark_dataset', 'FakeVideo-FakeAudio')




os.makedirs(os.path.join('debugging_dataset', 'RealVideo-RealAudio'), exist_ok=True)
os.makedirs(os.path.join('debugging_dataset', 'FakeVideo-FakeAudio'), exist_ok=True)
os.makedirs(os.path.join('debugging_dataset', 'FakeVideo-RealAudio'), exist_ok=True)
os.makedirs(os.path.join('debugging_dataset', 'RealVideo-FakeAudio'), exist_ok=True)



test_real_real = random.sample(os.listdir(real_real_dir), 5)

for file in test_real_real:
  shutil.copy(os.path.join(real_real_dir, file), os.path.join('debugging_dataset', 'RealVideo-RealAudio'))

test_real_fake = random.sample(os.listdir(real_fake_dir), 5)

for file in test_real_fake:
  shutil.copy(os.path.join(real_fake_dir, file), os.path.join('debugging_dataset', 'RealVideo-FakeAudio'))

test_fake_real = random.sample(os.listdir(fake_real_dir), 5)

for file in test_fake_real:
  shutil.copy(os.path.join(fake_real_dir, file), os.path.join('debugging_dataset', 'FakeVideo-RealAudio'))

test_fake_fake = random.sample(os.listdir(fake_fake_dir), 5)

for file in test_fake_fake:
  shutil.copy(os.path.join(fake_fake_dir, file), os.path.join('debugging_dataset', 'FakeVideo-FakeAudio'))


In [ ]:
#@title 1.2 Create Train and Test Directories

import os
import shutil
from sklearn.model_selection import train_test_split
from pathlib import Path

dataset_dir = Path('benchmark_dataset')

!rm -r test_dataset
!rm -r train_dataset

train_dir = dataset_dir.parent / 'train_dataset'
test_dir = dataset_dir.parent / 'test_dataset'

train_dir.mkdir(parents=True, exist_ok=True)
test_dir.mkdir(parents=True, exist_ok=True)

label_mapping = {
    'RealVideo-RealAudio': 0,
    'RealVideo-FakeAudio': 1,
    'FakeVideo-RealAudio': 1,
    'FakeVideo-FakeAudio': 1,
}

files = []
labels = []

# Collect files and labels
for class_name in label_mapping.keys():
    class_dir = dataset_dir / class_name
    for file in os.listdir(class_dir):
        if file.endswith(".mp4"):  # Assuming you're working with mp4 files
            files.append(class_dir / file)
            labels.append(label_mapping[class_name])

# Test train split
files_train, files_test, _, _ = train_test_split(files, labels, test_size=0.2, random_state=42)


def copy_files(files, destination):
    for file_path in files:
        # Construct the new path in the destination directory
        new_path = destination / file_path.relative_to(dataset_dir)
        new_path.parent.mkdir(parents=True, exist_ok=True)  # Create subdirectories if they don't exist
        shutil.copy(file_path, new_path)  # Copy the file to the new location

copy_files(files_train, train_dir)
copy_files(files_test, test_dir)


rm: cannot remove 'test_dataset': No such file or directory
rm: cannot remove 'train_dataset': No such file or directory


In [ ]:
# If comparing Xception, skip ahead to the marker 4.0.
# To measure control models, please continue

In [ ]:
count = 0
for root, dirs, files in os.walk('debugging_dataset'):
  for file in files:
    if file.endswith('.mp4'):
      count += 1
count

20

In [ ]:
# Remove models in current working directory:

def remove_old_files():
  !rm -r scaler_audio.pkl
  !rm -r scaler_video.pkl
  !rm -r scaler_combined.pkl
  !rm -r svm_model_combined.pkl
  !rm -r svm_model_audio.pkl
  !rm -r svm_model_video.pkl
  !rm -r mlp_audio_model.keras
  !rm -r mlp_video_model.keras
  !rm -r mlp_combined_model.keras

In [ ]:
#@title Streamlined code
import os
import time
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, fbeta_score, roc_curve, auc, recall_score, confusion_matrix, classification_report
import joblib
import librosa
import cv2
from skimage.feature import local_binary_pattern
from keras.models import Sequential, load_model
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_audio
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from keras.layers import Dense
from keras.optimizers import Adam
from sklearn.preprocessing import LabelEncoder, normalize
from keras.utils import to_categorical
import logging
import albumentations as A

def extract_audio_features(audio_path, n_mfcc=5, n_fft=2048, hop_length=512):
    try:
        audio_data, sr = librosa.load(audio_path, sr=None)
        mfccs = librosa.feature.mfcc(y=audio_data, sr=sr, n_mfcc=n_mfcc, n_fft=n_fft, hop_length=hop_length)

        S = librosa.magphase(librosa.stft(audio_data, n_fft=n_fft, hop_length=hop_length, window=np.ones, center=False))[0]
        rms = librosa.feature.rms(S=S, frame_length=n_fft, hop_length=hop_length)
        zero_crossing_rate = librosa.feature.zero_crossing_rate(y=audio_data, frame_length=n_fft, hop_length=hop_length)


        # Normalise features
        mfcc_norm = normalize(mfccs, norm='l2', axis=1)
        rms_norm = normalize(rms, norm='l2', axis=1)
        zcr_norm = normalize(zero_crossing_rate, norm='l2', axis=1)

        weight_rms = 2.0  # Increase RMS influence
        weight_zcr = 2.0  # Increase ZCR influence

        # Calculate statistical features: mean and standard deviation
        mfcc_mean = np.mean(mfcc_norm, axis=1)
        mfcc_std = np.std(mfcc_norm, axis=1)
        rms_mean = np.mean(rms_norm, axis=1) * weight_rms
        rms_std = np.std(rms_norm, axis=1) * weight_rms
        zcr_mean = np.mean(zcr_norm, axis=1) * weight_zcr
        zcr_std = np.std(zcr_norm, axis=1) * weight_zcr

        # Combine statistical features
        combined_features = np.hstack((mfcc_mean, mfcc_std, rms_mean, rms_std, zcr_mean, zcr_std))
        print(f'Audio Feature shape: {combined_features.shape}')
        return combined_features

    except Exception as e:
        print(f"Error processing audio file {audio_path}: {e}")
        return None

transform_pipeline = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.25, p=0.5),
    A.OneOf([
        A.MotionBlur(p=0.2),
        A.MedianBlur(blur_limit=3, p=0.1),
        A.GaussianBlur(blur_limit=3, p=0.1),
    ], p=0.2),
    A.ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.2, rotate_limit=15, p=0.5),
])


def extract_video_features(video_path, is_training=True, sample_count=50, max_attempts=5):
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    lbp_histograms = []

    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("Error: Could not open video.")
        return None

    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    attempts = 0
    detected = False

    while not detected and attempts < max_attempts:
        frame_indices = np.linspace(0, total_frames - 1, sample_count, dtype=int)

        for frame_index in frame_indices:
            cap.set(cv2.CAP_PROP_POS_FRAMES, frame_index)
            ret, frame = cap.read()
            if not ret:
                continue

            # Don't augment testing frames
            if is_training:
                augmented = transform_pipeline(image=frame)
                frame = augmented['image']

            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            faces = face_cascade.detectMultiScale(gray, 1.1, 4)

            if len(faces) > 0:
                detected = True  # At least one face was detected

            for (x, y, w, h) in faces:
                face_region = gray[y:y+h, x:x+w]
                lbp = local_binary_pattern(face_region, P=8, R=1, method="uniform")
                (lbp_hist, _) = np.histogram(lbp.ravel(), bins=np.arange(257), range=(0, 256))
                lbp_hist = lbp_hist.astype("float")
                lbp_hist /= (lbp_hist.sum() + 1e-7)
                lbp_histograms.append(lbp_hist)

        if not detected:
            print(f"No faces detected on attempt {attempts+1}. Increasing sampling rate.")
            sample_count = int(sample_count * 1.5)  # Increase sampling rate by 50%
            attempts += 1

    cap.release()
    if len(lbp_histograms) > 0:
        print(f'Video Feature shape: {np.array(lbp_histograms).shape}')
        return np.mean(lbp_histograms, axis=0)
    else:
        print("No facial features detected in the video.")
        return None

def load_data(root_dir, is_training):
    X_video, X_audio, y = [], [], []
    count = 0

    # Mapping directory names to labels
    label_mapping = {
        'RealVideo-RealAudio': 0,
        'RealVideo-FakeAudio': 1,
        'FakeVideo-RealAudio': 1,
        'FakeVideo-FakeAudio': 1,
    }

    for class_name in sorted(os.listdir(root_dir)):
        class_dir = os.path.join(root_dir, class_name)
        label = label_mapping.get(class_name, 0)  # Default to 0 (real) if not found

        for video_name in sorted(os.listdir(class_dir)):
            print(f'\tProcessing {video_name} ... ')
            video_path = os.path.join(class_dir, video_name)
            video_features = extract_video_features(video_path, is_training)
            if video_features is None:
              print(f'\tFOUND CORRUPTED VIDEO: {video_path}')
            if video_features is not None:
                X_video.append(video_features)  # No padding needed here

            audio_output_path = os.path.join("temp", video_name.split('.')[0] + ".wav")
            if not os.path.isfile(audio_output_path):
                ffmpeg_extract_audio(video_path, audio_output_path)
            audio_features = extract_audio_features(audio_output_path)
            if audio_features is not None:
                X_audio.append(audio_features)
                y.append(label)

            count += 1
            print(f'Processed {count} videos')
    return np.array(X_video, dtype=object), np.array(X_audio), np.array(y)


def train_mlp(X_video, X_audio, y, early_fusion=True):
    if early_fusion:
        X = np.concatenate((X_video, X_audio), axis=1)  # Early fusion by concatenating features
    else:
        X = X_video if X_video is not None else X_audio  # For late fusion, this function is called separately for video and audio

    encoder = LabelEncoder()
    encoder.fit(y)
    encoded_Y = encoder.transform(y)
    # Convert integers to dummy variables (i.e. one hot encoded)
    dummy_y = to_categorical(encoded_Y)

    model = Sequential()
    model.add(Dense(512, input_dim=X.shape[1], activation='relu'))
    model.add(Dense(256, activation='relu'))
    model.add(Dense(dummy_y.shape[1], activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.001), metrics=['accuracy'])

    model.fit(X, dummy_y, epochs=20, batch_size=32, verbose=1)

    return model


def train_svm(X_video, X_audio, y, early_fusion=True):
    if early_fusion:
        X = np.concatenate((X_video, X_audio), axis=1)  # Early fusion by concatenating features
    else:
        X = X_video if X_video is not None else X_audio  # For late fusion, this function is called separately for video and audio

    scaler = StandardScaler().fit(X)
    X_scaled = scaler.transform(X)

    model = SVC(C=1.0, kernel='rbf', probability=True)
    model.fit(X_scaled, y)

    return model, scaler


def scale_features(X_video, X_audio):
    scaler_video = StandardScaler()
    X_video_scaled = scaler_video.fit_transform(X_video)

    scaler_audio = StandardScaler()
    X_audio_scaled = scaler_audio.fit_transform(X_audio)

    return X_video_scaled, X_audio_scaled, scaler_video, scaler_audio

def evaluate_model(X_video_test, X_audio_test, y_test, model, scaler_video, scaler_audio, model_type="MLP", fusion_type="Early"):

    X_video_test_scaled = scaler_video.transform(X_video_test)
    print(f'Shape of X_video_test_scaled: {X_video_test_scaled.shape}')
    X_audio_test_scaled = scaler_audio.transform(X_audio_test)
    print(f'Shape of X_audio_test_scaled: {X_audio_test_scaled.shape}')

    if fusion_type == "Early":
        X_test_combined_scaled = np.concatenate((X_video_test_scaled, X_audio_test_scaled), axis=1)
        print(f'Shape of X test: {X_test_combined_scaled.shape}')
        if model_type == "MLP":
            combined_probs = model.predict(X_test_combined_scaled)[:, 1]
            print(f'Shape of combined_predictions: {combined_probs.shape}')
        else:  # SVM
            combined_probs = model.predict_proba(X_test_combined_scaled)[:, 1]
    else:  # Late Fusion
        if model_type == "MLP":
            video_probs = model[0].predict(X_video_test_scaled)[:, 1]
            audio_probs = model[1].predict(X_audio_test_scaled)[:, 1]
        else:  # SVM
            video_probs = model[0].predict_proba(X_video_test_scaled)[:, 1]
            audio_probs = model[1].predict_proba(X_audio_test_scaled)[:, 1]
        combined_probs = (video_probs + audio_probs) / 2

    combined_predictions = (combined_probs > 0.5).astype(int)

    # Evaluate and display results
    accuracy = accuracy_score(y_test, combined_predictions)
    f2_score = fbeta_score(y_test, combined_predictions, beta=2)
    roc_auc = auc(*roc_curve(y_test, combined_probs)[:2])

    print(f"Model Type: {model_type}, Fusion Type: {fusion_type}")
    print(f"Accuracy: {accuracy:.4f}, F2-Score: {f2_score:.4f}, ROC AUC: {roc_auc:.4f}")
    print(classification_report(y_test, combined_predictions, target_names=['real', 'deepfake']))

def main(train_dataset, test_dataset, model_type="MLP", fusion_type="Early"):
    remove_old_files()
    # Load and scale features
    X_video, X_audio, y = load_data(train_dataset, is_training=True)
    X_video_scaled, X_audio_scaled, scaler_video, scaler_audio = scale_features(X_video, X_audio)

    # Model training
    if model_type == "MLP":
        if fusion_type == "Early":
            model = train_mlp(X_video_scaled, X_audio_scaled, y, early_fusion=True)
        else:  # Late Fusion
            model_video = train_mlp(X_video_scaled, None, y, early_fusion=False)
            model_audio = train_mlp(None, X_audio_scaled, y, early_fusion=False)
            model = (model_video, model_audio)
    else:  # SVM
        if fusion_type == "Early":
            model, _ = train_svm(X_video_scaled, X_audio_scaled, y, early_fusion=True)
        else:  # Late Fusion
            model_video, _ = train_svm(X_video_scaled, None, y, early_fusion=False)
            model_audio, _ = train_svm(None, X_audio_scaled, y, early_fusion=False)
            model = (model_video, model_audio)

    # Evaluation
    start_time = time.monotonic()
    X_video_test, X_audio_test, y_test = load_data(test_dataset, is_training=False)
    evaluate_model(X_video_test, X_audio_test, y_test, model, scaler_video, scaler_audio, model_type, fusion_type)
    end_time = time.monotonic()
    print(f'Time Taken for Model to Evaluate on Test Dataset is {(end_time - start_time):.4f}')


/usr/local/lib/python3.10/dist-packages/albumentations/augmentations/blur/transforms.py:184: UserWarning: blur_limit and sigma_limit minimum value can not be both equal to 0. blur_limit minimum value changed to 3.
  warnings.warn(


In [ ]:

base__train__dir = "train_dataset"
base__test__dir = "test_dataset"
# Example usage: train and evaluate an MLP with Early Fusion
main(base__train__dir, base__test__dir, model_type="MLP", fusion_type="Early")
# To switch to another configuration, just call main() with different parameters
# main(model_type="SVM", fusion_type="Late")

rm: cannot remove 'scaler_audio.pkl': No such file or directory
rm: cannot remove 'scaler_video.pkl': No such file or directory
rm: cannot remove 'scaler_combined.pkl': No such file or directory
rm: cannot remove 'svm_model_combined.pkl': No such file or directory
rm: cannot remove 'svm_model_audio.pkl': No such file or directory
rm: cannot remove 'svm_model_video.pkl': No such file or directory
rm: cannot remove 'mlp_audio_model.keras': No such file or directory
rm: cannot remove 'mlp_video_model.keras': No such file or directory
rm: cannot remove 'mlp_combined_model.keras': No such file or directory
	Processing men_id00018_00181_id00529_wavtolip.mp4 ... 
Video Feature shape: (48, 256)
Audio Feature shape: (14,)
Processed 1 videos
	Processing men_id00029_00288_id00052_NK_leX996Yk_id00696_wavtolip.mp4 ... 
Video Feature shape: (51, 256)
Audio Feature shape: (14,)
Processed 2 videos
	Processing men_id00032_00028_1_id07108_wavtolip.mp4 ... 
Video Feature shape: (51, 256)
Audio Feature sh

Video Feature shape: (21, 256)
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
Audio Feature shape: (14,)
Processed 640 videos
Epoch 1/20
20/20 [==============================] - 1s 5ms/step - loss: 0.4515 - accuracy: 0.7703
Epoch 2/20
20/20 [==============================] - 0s 5ms/step - loss: 0.3327 - accuracy: 0.8469
Epoch 3/20
20/20 [==============================] - 0s 4ms/step - loss: 0.2693 - accuracy: 0.8922
Epoch 4/20
20/20 [==============================] - 0s 5ms/step - loss: 0.2206 - accuracy: 0.9016
Epoch 5/20
20/20 [==============================] - 0s 5ms/step - loss: 0.1897 - accuracy: 0.9234
Epoch 6/20
20/20 [==============================] - 0s 4ms/step - loss: 0.1639 - accuracy: 0.9406
Epoch 7/20
20/20 [==============================] - 0s 5ms/step - loss: 0.1513 - accuracy: 0.9344
Epoch 8/20
20/20 [==============================] - 0s 5ms/step - loss: 0.1243 - accuracy: 0.9563
Epoch 9/20
20/20 [==============================] - 0s 5ms/step - lo

In [ ]:
main(base__train__dir, base__test__dir, model_type="MLP", fusion_type="Late")


rm: cannot remove 'scaler_audio.pkl': No such file or directory
rm: cannot remove 'scaler_video.pkl': No such file or directory
rm: cannot remove 'scaler_combined.pkl': No such file or directory
rm: cannot remove 'svm_model_combined.pkl': No such file or directory
rm: cannot remove 'svm_model_audio.pkl': No such file or directory
rm: cannot remove 'svm_model_video.pkl': No such file or directory
rm: cannot remove 'mlp_audio_model.keras': No such file or directory
rm: cannot remove 'mlp_video_model.keras': No such file or directory
rm: cannot remove 'mlp_combined_model.keras': No such file or directory
	Processing men_id00018_00181_id00529_wavtolip.mp4 ... 
Video Feature shape: (47, 256)
Audio Feature shape: (14,)
Processed 1 videos
	Processing men_id00029_00288_id00052_NK_leX996Yk_id00696_wavtolip.mp4 ... 
Video Feature shape: (52, 256)
Audio Feature shape: (14,)
Processed 2 videos
	Processing men_id00032_00028_1_id07108_wavtolip.mp4 ... 
Video Feature shape: (48, 256)
Audio Feature sh

Video Feature shape: (20, 256)
Audio Feature shape: (14,)
Processed 640 videos
Epoch 1/20
20/20 [==============================] - 1s 5ms/step - loss: 0.5579 - accuracy: 0.7406
Epoch 2/20
20/20 [==============================] - 0s 5ms/step - loss: 0.4831 - accuracy: 0.7406
Epoch 3/20
20/20 [==============================] - 0s 5ms/step - loss: 0.4544 - accuracy: 0.7422
Epoch 4/20
20/20 [==============================] - 0s 5ms/step - loss: 0.4323 - accuracy: 0.7437
Epoch 5/20
20/20 [==============================] - 0s 5ms/step - loss: 0.4144 - accuracy: 0.7594
Epoch 6/20
20/20 [==============================] - 0s 5ms/step - loss: 0.4081 - accuracy: 0.7641
Epoch 7/20
20/20 [==============================] - 0s 4ms/step - loss: 0.4055 - accuracy: 0.7594
Epoch 8/20
20/20 [==============================] - 0s 4ms/step - loss: 0.3886 - accuracy: 0.7656
Epoch 9/20
20/20 [==============================] - 0s 4ms/step - loss: 0.3921 - accuracy: 0.7750
Epoch 10/20
20/20 [====================

Epoch 1/20
20/20 [==============================] - 1s 4ms/step - loss: 0.4773 - accuracy: 0.7516
Epoch 2/20
20/20 [==============================] - 0s 4ms/step - loss: 0.4172 - accuracy: 0.7672
Epoch 3/20
20/20 [==============================] - 0s 4ms/step - loss: 0.3977 - accuracy: 0.7875
Epoch 4/20
20/20 [==============================] - 0s 4ms/step - loss: 0.3837 - accuracy: 0.7922
Epoch 5/20
20/20 [==============================] - 0s 4ms/step - loss: 0.3689 - accuracy: 0.7984
Epoch 6/20
20/20 [==============================] - 0s 4ms/step - loss: 0.3628 - accuracy: 0.8062
Epoch 7/20
20/20 [==============================] - 0s 4ms/step - loss: 0.3535 - accuracy: 0.8328
Epoch 8/20
20/20 [==============================] - 0s 4ms/step - loss: 0.3442 - accuracy: 0.8250
Epoch 9/20
20/20 [==============================] - 0s 4ms/step - loss: 0.3413 - accuracy: 0.8281
Epoch 10/20
20/20 [==============================] - 0s 4ms/step - loss: 0.3276 - accuracy: 0.8422
Epoch 11/20
20/20 [

In [ ]:
main(base__train__dir, base__test__dir, model_type="SVM", fusion_type="Early")


rm: cannot remove 'scaler_audio.pkl': No such file or directory
rm: cannot remove 'scaler_video.pkl': No such file or directory
rm: cannot remove 'scaler_combined.pkl': No such file or directory
rm: cannot remove 'svm_model_combined.pkl': No such file or directory
rm: cannot remove 'svm_model_audio.pkl': No such file or directory
rm: cannot remove 'svm_model_video.pkl': No such file or directory
rm: cannot remove 'mlp_audio_model.keras': No such file or directory
rm: cannot remove 'mlp_video_model.keras': No such file or directory
rm: cannot remove 'mlp_combined_model.keras': No such file or directory
	Processing men_id00018_00181_id00529_wavtolip.mp4 ... 
Video Feature shape: (48, 256)
Audio Feature shape: (14,)
Processed 1 videos
	Processing men_id00029_00288_id00052_NK_leX996Yk_id00696_wavtolip.mp4 ... 
Video Feature shape: (47, 256)
Audio Feature shape: (14,)
Processed 2 videos
	Processing men_id00032_00028_1_id07108_wavtolip.mp4 ... 
Video Feature shape: (47, 256)
Audio Feature sh

In [ ]:
main(base__train__dir, base__test__dir, model_type="SVM", fusion_type="Late")


In [ ]:
#####################################

In [ ]:
main(base__train__dir, base__test__dir, model_type="SVM", fusion_type="Late")

rm: cannot remove 'scaler_audio.pkl': No such file or directory
rm: cannot remove 'scaler_video.pkl': No such file or directory
rm: cannot remove 'scaler_combined.pkl': No such file or directory
rm: cannot remove 'svm_model_combined.pkl': No such file or directory
rm: cannot remove 'svm_model_audio.pkl': No such file or directory
rm: cannot remove 'svm_model_video.pkl': No such file or directory
rm: cannot remove 'mlp_audio_model.keras': No such file or directory
rm: cannot remove 'mlp_video_model.keras': No such file or directory
rm: cannot remove 'mlp_combined_model.keras': No such file or directory
	Processing men_id00018_00181_id00529_wavtolip.mp4 ... 
Video Feature shape: (44, 256)
Audio Feature shape: (14,)
Processed 1 videos
	Processing men_id00029_00288_id00052_NK_leX996Yk_id00696_wavtolip.mp4 ... 
Video Feature shape: (49, 256)
Audio Feature shape: (14,)
Processed 2 videos
	Processing men_id00032_00028_1_id07108_wavtolip.mp4 ... 
Video Feature shape: (46, 256)
Audio Feature sh

In [ ]:
main(base__train__dir, base__test__dir, model_type="SVM", fusion_type="Early")

rm: cannot remove 'scaler_audio.pkl': No such file or directory
rm: cannot remove 'scaler_video.pkl': No such file or directory
rm: cannot remove 'scaler_combined.pkl': No such file or directory
rm: cannot remove 'svm_model_combined.pkl': No such file or directory
rm: cannot remove 'svm_model_audio.pkl': No such file or directory
rm: cannot remove 'svm_model_video.pkl': No such file or directory
rm: cannot remove 'mlp_audio_model.keras': No such file or directory
rm: cannot remove 'mlp_video_model.keras': No such file or directory
rm: cannot remove 'mlp_combined_model.keras': No such file or directory
	Processing men_id00018_00181_id00529_wavtolip.mp4 ... 
Video Feature shape: (45, 256)
Audio Feature shape: (14,)
Processed 1 videos
	Processing men_id00029_00288_id00052_NK_leX996Yk_id00696_wavtolip.mp4 ... 
Video Feature shape: (51, 256)
Audio Feature shape: (14,)
Processed 2 videos
	Processing men_id00032_00028_1_id07108_wavtolip.mp4 ... 
Video Feature shape: (44, 256)
Audio Feature sh

In [ ]:
main(base__train__dir, base__test__dir, model_type="MLP", fusion_type="Early")

rm: cannot remove 'scaler_audio.pkl': No such file or directory
rm: cannot remove 'scaler_video.pkl': No such file or directory
rm: cannot remove 'scaler_combined.pkl': No such file or directory
rm: cannot remove 'svm_model_combined.pkl': No such file or directory
rm: cannot remove 'svm_model_audio.pkl': No such file or directory
rm: cannot remove 'svm_model_video.pkl': No such file or directory
rm: cannot remove 'mlp_audio_model.keras': No such file or directory
rm: cannot remove 'mlp_video_model.keras': No such file or directory
rm: cannot remove 'mlp_combined_model.keras': No such file or directory
	Processing men_id00018_00181_id00529_wavtolip.mp4 ... 
Video Feature shape: (48, 256)
Audio Feature shape: (14,)
Processed 1 videos
	Processing men_id00029_00288_id00052_NK_leX996Yk_id00696_wavtolip.mp4 ... 
Video Feature shape: (50, 256)
Audio Feature shape: (14,)
Processed 2 videos
	Processing men_id00032_00028_1_id07108_wavtolip.mp4 ... 
Video Feature shape: (46, 256)
Audio Feature sh

Video Feature shape: (18, 256)
Audio Feature shape: (14,)
Processed 640 videos
Epoch 1/20
20/20 [==============================] - 1s 4ms/step - loss: 0.4468 - accuracy: 0.7812
Epoch 2/20
20/20 [==============================] - 0s 5ms/step - loss: 0.3268 - accuracy: 0.8516
Epoch 3/20
20/20 [==============================] - 0s 4ms/step - loss: 0.2620 - accuracy: 0.8781
Epoch 4/20
20/20 [==============================] - 0s 4ms/step - loss: 0.2187 - accuracy: 0.9109
Epoch 5/20
20/20 [==============================] - 0s 5ms/step - loss: 0.1789 - accuracy: 0.9250
Epoch 6/20
20/20 [==============================] - 0s 4ms/step - loss: 0.1616 - accuracy: 0.9297
Epoch 7/20
20/20 [==============================] - 0s 4ms/step - loss: 0.1344 - accuracy: 0.9484
Epoch 8/20
20/20 [==============================] - 0s 4ms/step - loss: 0.1332 - accuracy: 0.9438
Epoch 9/20
20/20 [==============================] - 0s 4ms/step - loss: 0.1228 - accuracy: 0.9438
Epoch 10/20
20/20 [====================

In [ ]:
main(base__train__dir, base__test__dir, model_type="MLP", fusion_type="Late")

rm: cannot remove 'scaler_audio.pkl': No such file or directory
rm: cannot remove 'scaler_video.pkl': No such file or directory
rm: cannot remove 'scaler_combined.pkl': No such file or directory
rm: cannot remove 'svm_model_combined.pkl': No such file or directory
rm: cannot remove 'svm_model_audio.pkl': No such file or directory
rm: cannot remove 'svm_model_video.pkl': No such file or directory
rm: cannot remove 'mlp_audio_model.keras': No such file or directory
rm: cannot remove 'mlp_video_model.keras': No such file or directory
rm: cannot remove 'mlp_combined_model.keras': No such file or directory
	Processing men_id00018_00181_id00529_wavtolip.mp4 ... 
Video Feature shape: (49, 256)
Audio Feature shape: (14,)
Processed 1 videos
	Processing men_id00029_00288_id00052_NK_leX996Yk_id00696_wavtolip.mp4 ... 
Video Feature shape: (49, 256)
Audio Feature shape: (14,)
Processed 2 videos
	Processing men_id00032_00028_1_id07108_wavtolip.mp4 ... 
Video Feature shape: (47, 256)
Audio Feature sh

Video Feature shape: (15, 256)
Audio Feature shape: (14,)
Processed 640 videos
Epoch 1/20
20/20 [==============================] - 1s 4ms/step - loss: 0.5311 - accuracy: 0.7437
Epoch 2/20
20/20 [==============================] - 0s 4ms/step - loss: 0.4715 - accuracy: 0.7500
Epoch 3/20
20/20 [==============================] - 0s 4ms/step - loss: 0.4498 - accuracy: 0.7312
Epoch 4/20
20/20 [==============================] - 0s 4ms/step - loss: 0.4237 - accuracy: 0.7437
Epoch 5/20
20/20 [==============================] - 0s 4ms/step - loss: 0.4056 - accuracy: 0.7578
Epoch 6/20
20/20 [==============================] - 0s 4ms/step - loss: 0.4045 - accuracy: 0.7625
Epoch 7/20
20/20 [==============================] - 0s 4ms/step - loss: 0.4058 - accuracy: 0.7656
Epoch 8/20
20/20 [==============================] - 0s 4ms/step - loss: 0.3927 - accuracy: 0.7828
Epoch 9/20
20/20 [==============================] - 0s 5ms/step - loss: 0.3902 - accuracy: 0.7828
Epoch 10/20
20/20 [====================

Epoch 1/20
20/20 [==============================] - 1s 4ms/step - loss: 0.4960 - accuracy: 0.7297
Epoch 2/20
20/20 [==============================] - 0s 3ms/step - loss: 0.4180 - accuracy: 0.7641
Epoch 3/20
20/20 [==============================] - 0s 4ms/step - loss: 0.4023 - accuracy: 0.7844
Epoch 4/20
20/20 [==============================] - 0s 3ms/step - loss: 0.3963 - accuracy: 0.7984
Epoch 5/20
20/20 [==============================] - 0s 4ms/step - loss: 0.3812 - accuracy: 0.8016
Epoch 6/20
20/20 [==============================] - 0s 4ms/step - loss: 0.3690 - accuracy: 0.8125
Epoch 7/20
20/20 [==============================] - 0s 4ms/step - loss: 0.3591 - accuracy: 0.8156
Epoch 8/20
20/20 [==============================] - 0s 4ms/step - loss: 0.3557 - accuracy: 0.8219
Epoch 9/20
20/20 [==============================] - 0s 4ms/step - loss: 0.3408 - accuracy: 0.8250
Epoch 10/20
20/20 [==============================] - 0s 4ms/step - loss: 0.3326 - accuracy: 0.8328
Epoch 11/20
20/20 [